In [ ]:
import cvxpy as cp
import numpy as np

profit = cp.Variable(41, nonneg = True) # Profit of movie i
tickets = cp.Variable(41, integer = True) # Number of tickets sold for movie (i) at timeslot (j)
movie = cp.Variable(8, integer = True) # Number of movie i shown that day
theater1 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 1 at time (j)
theater2 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 2 at time (j)
theater3 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 3 at time (j)
theater4 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 4 at time (j)
theater5 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 5 at time (j)
theater6 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 6 at time (j)
theater7 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 7 at time (j)
theater8 = cp.Variable([8, 41], boolean = True) # Describes if movie (i) is being shown in theater 8 at time (j)

# Movies to be shown at specific timeslots
movies = ["Gladiator 2", "Anora", "Wicked", "A Real Pain", "Life is Beautiful", "Heretic", "Basquiat", "American Werewolf in London"]

# Movie popularity index, which is derived from current position on charts
# If movie is not currently on charts or was released long ago, use approximate based on peak
mPopularity = [1, 93.0/100, 94.0/100, 66.0/100, 73.0/100, 95.0/100, 25.0/100, 15.0/100] # Popularity of movie (i)

# Length of movie
movieLength = [10, 10, 11, 6, 8, 7, 7, 6]

# Capacity of each theater, found from the gateway website for ticketing
theaterCapacity = [113, 113, 127, 312, 72, 72, 113, 44]

# The popularity of each timeslot, which was found via google analytics and reviews
tPopularity = np.array([0.07, 0.07, 0.07, 0.07, 0.2, 0.2, 0.2, 0.2, 0.35, 0.35, 0.35, 0.35, 0.45, 0.45, 0.45, 0.45, 0.6, 0.6, 0.6, 0.6, 0.75, 
               0.75, 0.75, 0.75, 0.85, 0.85, 0.85, 0.85, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.8]) # Popularity of timeslot (j)

# Approximate the number of people based on movie theater size, then divide by time slot (since by hour)
# Round to whole number since can't have .5 of a person
tPeople = np.round((966/4) * tPopularity, 0)

# Price vector of each timeslot
price = [9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 9.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75,
         13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50, 13.50]

# Timeslot matrix
timeslot = [0 + 15 * i for i in range(41)]

constraints = []

for i in range (8):
    # My idea to constrain movies.  Still not working with time slots though
    constraints.append(sum(theater1[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater2[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater3[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater4[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater5[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater6[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater7[i]) <= 41.0/movieLength[i])
    constraints.append(sum(theater8[i]) <= 41.0/movieLength[i])
    constraints.append(movie[i] >= 1)
    constraints.append(movie[i] <= 5)
    constraints.append(sum(theater1[i]) + sum(theater2[i]) + sum(theater3[i]) + sum(theater4[i]) + 
                       sum(theater5[i]) + sum(theater6[i]) + sum(theater7[i]) + sum(theater8[i]) == movie[i])
    
    for j in range (41):
        constraints.append(profit[j] == tickets[j] * price[j])

        # Multiplies together each theater capacity by the boolean theater value, populating a matrix with potential filled
        # theater seats by movie.  Then, multiplies these values by the popularity of the movie and the popularity of the time.  
        # TODO: Figure out why multiplying in movie popularity breaks everything
        constraints.append(tickets[j] == (theaterCapacity[0] * (theater1[i, j]) + theaterCapacity[1] * (theater2[i, j]) 
                           + theaterCapacity[2] * (theater3[i, j]) + theaterCapacity[3] * (theater4[i, j]) + 
                           theaterCapacity[4] * (theater5[i, j]) +  theaterCapacity[5] * (theater6[i, j]) + 
                           theaterCapacity[6] * (theater7[i, j]) +  theaterCapacity[7] * (theater8[i, j])) * tPopularity[j])
        
        


obj_func = sum(profit)
problem = cp.Problem(cp.Maximize(obj_func), constraints)

problem.solve(solver=cp.GUROBI,verbose = True)

print("optimal objective value =")
print(obj_func.value)
print("profit =")
print(profit.value)
print("tickets =")
print(tickets.value)
print("theater =")
print(theater1.value)

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 18 11:23:23 PM: Your problem has 2714 variables, 744 constraints, and 0 parameters.
(CVXPY) Nov 18 11:23:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 18 11:23:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 18 11:23:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 18 11:23:23 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 18 11:23:23 PM: Compiling problem (target solver=GUROBI)